In [1]:
# import itertools

# import torch
# from torch.utils.data import Dataset

# import sys
# sys.path.append("..")
# from common.sentence import Sentence
# from common.parse_conllu import parse_conllu_incr

# from lemmatize_helper import predict_lemma_rule
# from utils import pad_matrices


# class CobaldDataset(Dataset):
#     # Alias for map from string to integer label.
#     type Vocabulary = dict[str, int]

#     def __init__(self, conllu_file_path: str, vocabularies: dict[str, Vocabulary] = None):
#         fields = []
#         with open(conllu_file_path, "r") as file:
#             for sentence in parse_conllu_incr(file):
#                 sentence_fields = self._process(sentence)
#                 fields.append(sentence_fields)

#         if vocabularies is None:
#             vocabularies = self._build_vocabularies(fields)
#         self._vocabularies = vocabularies

#         self._fields_encoded = [self._encode_field(field, vocabularies) for field in fields]

#     def vocabularies(self) -> dict[str, Vocabulary]:
#         return self._vocabularies

#     def __len__(self):
#         return len(self._fields_encoded)

#     def __getitem__(self, index):
#         return self._fields_encoded[index]

#     @staticmethod
#     def _process(sentence: Sentence) -> dict[str, list[str]]:
#         words = sentence.words
#         lemmas = sentence.lemmas
#         upos = sentence.upos
#         xpos = sentence.xpos
#         feats = sentence.feats
#         heads = sentence.heads
#         deprels = sentence.deprels
#         deps = sentence.deps
#         miscs = sentence.miscs
#         deepslots = sentence.semslots
#         semclasses = sentence.semclasses
#         metadata = sentence.metadata

#         lemma_rules: list[str] = None
#         if lemmas is not None:
#             lemma_rules = [
#                 str(predict_lemma_rule(word if word is not None else '', lemma if lemma is not None else ''))
#                 for word, lemma in zip(words, lemmas, strict=True)
#             ]

#         joint_pos_feats: list[str] = None
#         if upos is not None and xpos is not None and feats is not None:
#             joint_feats = [
#                 '|'.join([f"{k}={v}" for k, v in feat.items()]) if 0 < len(feat) else '_'
#                 for feat in feats
#             ]
#             joint_pos_feats = [
#                 f"{token_upos}#{token_xpos}#{token_joint_feats}"
#                 for token_upos, token_xpos, token_joint_feats in zip(upos, xpos, joint_feats, strict=True)
#             ]

#         sequence_length = len(words)

#         deps_matrix_ud = None
#         if heads is not None and deprels is not None:
#             deps_matrix_ud = [[''] * sequence_length for _ in range(sequence_length)]
#             for index, (head, relation) in enumerate(zip(heads, deprels, strict=True)):
#                 # Skip nulls.
#                 if head == -1:
#                     continue
#                 assert 0 <= head
#                 # Hack: start indexing at 0 and replace ROOT with self-loop.
#                 # It makes parser implementation much easier.
#                 if head == 0:
#                     # Replace ROOT with self-loop.
#                     head = index
#                 else:
#                     # If not ROOT, shift token left.
#                     head -= 1
#                     assert head != index, f"head = {head + 1} must not be equal to index = {index + 1}"
#                 deps_matrix_ud[index][head] = relation

#         deps_matrix_eud = None
#         if deps is not None:
#             deps_matrix_eud = [[''] * sequence_length for _ in range(sequence_length)]
#             for index, dep in enumerate(deps):
#                 assert 0 < len(dep), f"Deps must not be empty"
#                 for head, relation in dep.items():
#                     assert 0 <= head
#                     # Hack: start indexing at 0 and replace ROOT with self-loop.
#                     # It makes parser implementation much easier.
#                     if head == 0:
#                         # Replace ROOT with self-loop.
#                         head = index
#                     else:
#                         # If not ROOT, shift token left.
#                         head -= 1
#                         assert head != index, f"head = {head + 1} must not be equal to index = {index + 1}"
#                     deps_matrix_eud[index][head] = relation

#         return {
#             "words": words,
#             "lemma_rules": lemma_rules,
#             "joint_pos_feats": joint_pos_feats,
#             "deps_ud": deps_matrix_ud,
#             "deps_eud": deps_matrix_eud,
#             "miscs": miscs,
#             "deepslots": deepslots,
#             "semclasses": semclasses,
#             "metadata": metadata
#         }

#     @staticmethod
#     def _build_vocabularies(fields: list[dict]) -> dict[str, Vocabulary]:
        
#         def enumerate_column(fields, column: str, is_matrix = False) -> list[str]:
#             """Extract unique labels from a specified column from fields."""
#             if is_matrix:
#                 all_labels = [value for field in fields for matrix in field[column] for value in matrix]
#             else:
#                 all_labels = [value for field in fields for value in field[column]]
#             # Ensure consistent ordering of labels
#             unique_labels = sorted(set(all_labels))
#             return {label: i for i, label in enumerate(unique_labels)}

#         return {
#             "lemma_rules": enumerate_column(fields, "lemma_rules"),
#             "joint_pos_feats": enumerate_column(fields, "joint_pos_feats"),
#             "deps_ud": enumerate_column(fields, "deps_ud", is_matrix=True),
#             "deps_eud": enumerate_column(fields, "deps_eud", is_matrix=True),
#             "miscs": enumerate_column(fields, "miscs"),
#             "deepslots": enumerate_column(fields, "deepslots"),
#             "semclasses": enumerate_column(fields, "semclasses")
#         }

#     @staticmethod
#     def _encode_field(field: dict[str, list], vocabularies: dict[str, Vocabulary]) -> dict:
#         # print(field)
#         encode_sequence_column = lambda column: torch.tensor(
#             [vocabularies[column][val] for val in field[column]]
#         )
#         encode_matrix_column = lambda column: torch.tensor(
#             [[vocabularies[column][val] for val in row] for row in field[column]]
#         )
#         return {
#             "lemma_rules": encode_sequence_column("lemma_rules"),
#             "joint_pos_feats": encode_sequence_column("joint_pos_feats"),
#             "deps_ud": encode_matrix_column("deps_ud"),
#             "deps_eud": encode_matrix_column("deps_eud"),
#             "miscs": encode_sequence_column("miscs"),
#             "deepslots": encode_sequence_column("deepslots"),
#             "semclasses": encode_sequence_column("semclasses")
#         }


In [1]:
from dataset import CobaldJointDataset

train_dataset = CobaldJointDataset("../data/train.conllu")
train_dataset[0]

{'words': ['The',
  'firm',
  "'s",
  'snowmobile',
  'division',
  'and',
  'defence',
  'services',
  'unit',
  'were',
  'also',
  'sold',
  'and',
  'Bombardier',
  'started',
  'the',
  'development',
  'of',
  'a',
  'new',
  'aircraft',
  'seating',
  '110',
  'to',
  '135',
  'passengers',
  '.'],
 'lemma_rules': ['cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=1|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=1|cut_suffix=2|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=3|append_suffix=ell',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=0|append_suffix=',
  'cut_prefix=0|cut_suffix=2|ap

In [2]:
from vocabulary import CobaldJointVocabulary

vocabulary = CobaldJointVocabulary(train_dataset)
vocabulary.semclass_tagset

{'ACCESSORY': 0,
 'ACT': 1,
 'ACTIVITY': 2,
 'ACTIVITY_BY_INTEREST': 3,
 'ADMINISTRATIVE_REGION': 4,
 'ADVENTURE': 5,
 'AGGREGATE': 6,
 'AGGREGATE_OF_LIVING_OBJECTS': 7,
 'AGGREGATE_OF_MACHINERY_OR_TRANSPORT': 8,
 'AGGRESSIVE_ACTIONS': 9,
 'AGREEMENT_VERBS': 10,
 'AGRICULTURAL_PROCESSING': 11,
 'AMBIENCE_ENVIRONMENT': 12,
 'ANIMAL': 13,
 'APPARATUS': 14,
 'AREA_OF_HUMAN_ACTIVITY': 15,
 'ARRANGEMENTS': 16,
 'ARTEFACT': 17,
 'ARTICLES': 18,
 'ATTRIBUTIVE': 19,
 'AUXILIARY_VERBS': 20,
 'BAD_DANGEROUS_EVENT': 21,
 'BE': 22,
 'BEGIN_TO_TAKE_PLACE': 23,
 'BEHAVIOUR': 24,
 'BEING': 25,
 'BEVERAGE': 26,
 'BE_STATE': 27,
 'BIJOUTERIE_AND_JEWELLERY': 28,
 'BODY': 29,
 'BOOM': 30,
 'BUSINESS': 31,
 'BUSY_FREE_OCCUPIED': 32,
 'CARGO': 33,
 'CHANGE_OF_MATTER_PHYSICAL_STATE': 34,
 'CHANGE_OF_ORGANIC_OBJECTS': 35,
 'CHANGE_OF_POST_AND_JOB': 36,
 'CHARACTERISTIC_GENERAL': 37,
 'CHEMICAL_CHANGES': 38,
 'CHOOSING_SORTING': 39,
 'CH_ABSTRACT_GENERALIZED': 40,
 'CH_APPEARANCE': 41,
 'CH_ASPECT': 42,
 'CH_

In [3]:
from dataset import NO_ARC_LABEL

PADDING_VALUE = -1

# Little hack to easy following pipeline
vocabulary.deprel_ud_tagset[NO_ARC_LABEL] = PADDING_VALUE
vocabulary.deprel_eud_tagset[NO_ARC_LABEL] = PADDING_VALUE

In [4]:
transform = lambda sample: vocabulary.encode(sample)

train_dataset = CobaldJointDataset("../data/train.conllu", transform)
val_dataset = CobaldJointDataset("../data/validation.conllu", transform)
test_dataset = CobaldJointDataset("../data/test_clean.conllu", transform)

In [5]:
train_dataset[0]

{'words': ['The',
  'firm',
  "'s",
  'snowmobile',
  'division',
  'and',
  'defence',
  'services',
  'unit',
  'were',
  'also',
  'sold',
  'and',
  'Bombardier',
  'started',
  'the',
  'development',
  'of',
  'a',
  'new',
  'aircraft',
  'seating',
  '110',
  'to',
  '135',
  'passengers',
  '.'],
 'lemma_rules': tensor([  0,   0,   0,   0,   0,   0,   0,  13,   0, 143,   0,  74,   0,   0,
          32,   0,   0,   0,   0,   0,   0,  63,   0,   0,   0,  13,   0]),
 'joint_pos_feats': tensor([ 53,  79,  96,  79,  79,  50,  79,  78,  79,  34,  21, 192,  50, 152,
         179,  53,  79,  12,  54,   2,  79, 195,  85,  12,  85,  78, 160]),
 'deps_ud': tensor([[-1, 20, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, 34, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1],
         [-1, 11, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

In [6]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(train_dataset, batch_size=2, collate_fn=CobaldJointDataset.collate_fn, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, collate_fn=CobaldJointDataset.collate_fn, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, collate_fn=CobaldJointDataset.collate_fn, shuffle=False)

for batch in train_dataloader:
    if batch['deps_ud'].shape[-1] < 10:
        print(batch)
        break

{'words': [['However', ',', 'the', 'DTI', 'has', 'dismissed', 'the', 'proposal', '.'], ['We', 'are', 'taken', 'as', 'hostages', '.']], 'lemma_rules': tensor([[  0,   0,   0,   0,  27,  32,   0,   0,   0],
        [  0, 147,  13,   0,  13,   0,  -1,  -1,  -1]]), 'joint_pos_feats': tensor([[ 21, 160,  53, 150,  41, 191,  53,  79, 160],
        [124,  32, 192, 163,  78, 160,  -1,  -1,  -1]]), 'deps_ud': tensor([[[-1, -1, -1, -1, -1, 46, -1, -1, -1],
         [47, -1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, 20, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, 36, -1, -1, -1],
         [-1, -1, -1, -1, -1,  9, -1, -1, -1],
         [-1, -1, -1, -1, -1, 48, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, 20, -1],
         [-1, -1, -1, -1, -1, 41, -1, -1, -1],
         [-1, -1, -1, -1, -1, 47, -1, -1, -1]],

        [[-1, -1, 38, -1, -1, -1, -1, -1, -1],
         [-1, -1, 10, -1, -1, -1, -1, -1, -1],
         [-1, -1, 48, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, 11, 

In [9]:
import torch
import torch.nn.functional as F

# vocabulary = vocabulary.deprel_ud_tagset[

# for  in train_dataloader:
#     if sample['deps_ud'].shape[0] < 15:
#         break

# target = sample['deps_ud']
# target.argmax

# values, indices = batch['deps_eud'][0].max(dim=-1)
# mask = (values != -100)
# indices

target = batch['deps_eud']

print(target)

has_arc_mask = (target != -1)
# # target[has_arc_mask]

# s_arc = torch.rand(has_arc_mask.shape)
mask = torch.any(has_arc_mask == True, dim=-1)
print(mask)

# arc_losses = F.binary_cross_entropy_with_logits(s_arc[mask], has_arc_mask.float(), reduction='none')

tensor([[[ -1,  -1,  -1,  -1,  -1, 197,  -1,  -1,  -1],
         [198,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  -1,  55,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1, 120,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1,  40,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1, 200,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  55,  -1],
         [ -1,  -1,  -1,  -1,  -1, 126,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1, 198,  -1,  -1,  -1]],

        [[ -1,  -1, 122,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  41,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1, 200,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  42,  -1,  -1,  -1,  -1],
         [ -1,  -1, 139,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1, 198,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
         [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [71]:
s_rels = torch.rand([*target.shape, 7])
pred_rels = s_rels.argmax(-1)
(F.one_hot(pred_rels) * target_arcs[..., None]).nonzero()
# replace_masked_values(pred_rels * target_arcs)
# s_rels.shape

tensor([[0, 2, 1],
        [1, 2, 5],
        [2, 2, 0],
        [3, 2, 1]])

In [40]:
from importlib import reload  # Python 3.4+
import dependency_classifier
import metrics
metrics = reload(metrics)
dependency_classifier = reload(dependency_classifier)

depcls = dependency_classifier.DependencyHead(128, 300)

batch_size, seq_len, hid_dim = 2, 9, 128
h_arc_head = torch.rand((batch_size, seq_len, hid_dim))
h_arc_dep = torch.rand((batch_size, seq_len, hid_dim))
h_rel_head = torch.rand((batch_size, seq_len, hid_dim))
h_rel_dep = torch.rand((batch_size, seq_len, hid_dim))

depcls(h_arc_head, h_arc_dep, h_rel_head, h_rel_dep, target != -1, target, mask)

{'predictions': tensor([[  0,   0,   3,   3],
         [  0,   1,   4, 263],
         [  0,   2,   3, 278],
         [  0,   3,   3, 122],
         [  0,   4,   3, 184],
         [  0,   5,   4, 223],
         [  0,   6,   4,   3],
         [  0,   7,   3, 122],
         [  0,   8,   3, 157],
         [  1,   0,   0, 218],
         [  1,   1,   1, 157],
         [  1,   2,   5, 157],
         [  1,   3,   3, 157],
         [  1,   4,   3, 157],
         [  1,   5,   1,  43]]),
 'arc_loss': tensor(3.1764, grad_fn=<NllLossBackward0>),
 'rel_loss': tensor(5.6563, grad_fn=<NllLossBackward0>),
 'euas': 0.0,
 'elas': 0.0}